In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from pathlib import Path
from sklearn.neural_network import MLPRegressor
import numpy as np
def preprocess_data(filepath, items_id, Date):
    data = pd.read_csv(filepath)
    data['sales'].fillna(data['sales'].mean(), inplace=True)
    data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')

    # Filtering data based on itemID and Date if needed
    data = data[(data['sku_id'] == items_id) & (data['date'] == pd.to_datetime(Date, format='%d-%m-%Y'))]

    categorical_features = ['gender', 'category', 'brand', 'collection', 'price_tier', 'style']
    encoder = OneHotEncoder()
    encoded_categorical = encoder.fit_transform(data[categorical_features])
    encoded_features = pd.DataFrame(encoded_categorical.toarray(), columns=encoder.get_feature_names_out(categorical_features))
    data = pd.concat([data.drop(categorical_features, axis=1), encoded_features], axis=1)
    return data

def RecommendPrice(items_id, data_path, Time, Date):
    data = preprocess_data(data_path, items_id, Date)
    if data.empty:
        return "No data available for this item on the given date."

    # Select the features and check for NaN values
    X = data[['cost', 'price']]
    if X.isnull().any().any():
        X.fillna(method='ffill', inplace=True)

    y = data['sales']
    if y.isnull().any():
        y.fillna(y.mean(), inplace=True)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=2000)
    model.fit(X_scaled, y)

    prices = np.linspace(data['price'].min(), data['price'].max(), num=100)
    profits = []

    for price in prices:
        input_df = pd.DataFrame({
            'cost': X['cost'],
            'price': np.full(len(X), price)
        })
        input_scaled = scaler.transform(input_df)
        predicted_sales = model.predict(input_scaled)
        profit = (price * predicted_sales - X['cost']).sum()
        profits.append(profit)

    optimal_price = prices[np.argmax(profits)]
    return optimal_price


In [2]:
print('=====================')
data_path = 'data.csv'
test_samples_item_id = [{'items_id':[0], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[50], 'date': '24-05-2020', 'time':'8:00:00'},
                   {'items_id':[100], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[125], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[175], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[200], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[250], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[300], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[350], 'date': '25-06-2020', 'time':'8:00:00'},
                   {'items_id':[399], 'date': '25-06-2020', 'time':'8:00:00'}]

def test_single_sample(sample, data_path):
    items_id = sample['items_id']
    date = sample['date']
    time = sample['time']

    results = []
    for i in range(len(items_id)):
        cur_items_id = items_id[i]
        cur_res = RecommendPrice(cur_items_id, data_path, time, date)
        results.append(cur_res)
    return results

def compute_performance(test_samples, data_path):
    raw_res_list = []
    for cur_sample in test_samples:
        cur_res = test_single_sample(cur_sample, data_path)
        raw_res_list.append(cur_res)

    return raw_res_list

def test():
    results = compute_performance(test_samples_item_id, data_path)
    print('============================')
    print('results= ', results)


test()

results=  [[2.6], [1.2], [2.6], [2.6], [2.6], [2.6], [2.6], [2.6], [2.6], [2.6]]
